Group 35 Project Proposal

Group Memebers: Kyla, Melika, Henry, Kokoro

Context: Amongst others prevalent diseases, heart disease is one of the dealiest in the world being responsible for 16% of the world's total deaths since 2000. Risk factors for heart disease, include aging, high blood pressure, cholesterol and even stress can increase the risk of having this disease. Other than the high mortality rate, heart disease also attacks the individuals financially causing a financial burdern to their family and future plans.

Content: Using the dataset and targetted risk factors (listed below), we are going to analyze and predict which individuals are most likely to be diagnosed with heart disease.

Question: Does having the targetted risk factors (listed below), increase the chances of an individual getting diagnosed with heart disease?

Targetted Risk Factors: age, chest pain, resting blood pressure, cholesterol, blood surgar, depression, and maxium heart rate.

Data Set Information: The data set was daat collected during the 1988 with four databases: Cleveland, Hungary, Switzerland, and Long Beach. The data set is dowloded fro, kaggle.com (https://www.kaggle.com/johnsmith88/heart-disease-dataset) with the originally set containing 76 attributes. In the dataset downloaded, we are only using 14 of the attributes and only analyzing 7 varibles that could affect the diagnosis of heart disease.

In [19]:
library(tidyverse)

In [28]:
heart_data <- read_csv("heart_data.csv")

heart_data <- rename(heart_data,
                    chest_pain = cp,
                    blood_pressure = trestbps,
                    cholesterol = chol,
                    blood_sugar = fbs,
                    depression = oldpeak,
                    max_heart_rate = thalach)

heart_data_selected <- select(heart_data, age, chest_pain, blood_pressure, cholesterol, blood_sugar, depression, max_heart_rate)
heart_data_selected

Parsed with column specification:
cols(
  age = col_double(),
  sex = col_double(),
  cp = col_double(),
  trestbps = col_double(),
  chol = col_double(),
  fbs = col_double(),
  restecg = col_double(),
  thalach = col_double(),
  exang = col_double(),
  oldpeak = col_double(),
  slope = col_double(),
  ca = col_double(),
  thal = col_double(),
  target = col_double()
)



age,chest_pain,blood_pressure,cholesterol,blood_sugar,depression,max_heart_rate
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
63,3,145,233,1,2.3,150
37,2,130,250,0,3.5,187
41,1,130,204,0,1.4,172
56,1,120,236,0,0.8,178
57,0,120,354,0,0.6,163
57,0,140,192,0,0.4,148
56,1,140,294,0,1.3,153
44,1,120,263,0,0.0,173
52,2,172,199,1,0.5,162
